In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
from absl import flags
import numpy as np

import json
import cv2

import skimage.io as io
import tensorflow as tf

from src.util import depthRenderer as vis_util
from src.util import image as img_util
from src.util import openpose as op_util
import src.config
from src.RunModel import RunModel

In [2]:
flags.DEFINE_string('img_path', 'data/im1963.jpg', 'Image to run')
flags.DEFINE_string(
    'json_path', None,
    'If specified, uses the openpose output to crop the image.')

In [3]:
def visualize(img, proc_param, joints, verts, cam, img_path):
    """
    Renders the result in original image coordinate frame.
    """
    cam_for_render, vert_shifted, joints_orig = vis_util.get_original(
        proc_param, verts, cam, joints, img_size=img.shape[:2])

    folder = '/'.join(img_path.split('/')[0:-1])
    print("FOLDER!!!!!!!!!!!")
    print(folder)

#     # Render results
#     config = flags.FLAGS
#     config(sys.argv)
#     # Using pre-trained model, change this to use your own.
#     config.load_path = src.config.PRETRAINED_MODEL

#     config.batch_size = 1
    renderer = vis_util.SMPLRenderer(face_path=config.smpl_face_path)

    # skel_img = vis_util.draw_skeleton(img, joints_orig)
    # rend_img_overlay = renderer(
        # vert_shifted, cam=cam_for_render, img=img, do_alpha=True)
    rend_img = renderer(
        vert_shifted, cam=cam_for_render, img_size=img.shape[:2])
    # rend_img_vp1 = renderer.rotated(
    #     vert_shifted, 60, cam=cam_for_render, img_size=img.shape[:2])
    rend_img_vp2 = renderer.rotated(
        vert_shifted, 180, cam=cam_for_render, img_size=img.shape[:2])

    depthMapPath = folder + '/depthMap.png'
    print("Saving Depth Map to:")
    print(depthMapPath)

    cv2.imwrite(depthMapPath, rend_img)
    # cv2.imshow('Depth Map',rend_img)        
    # cv2.waitKey(0)

    depthMapPath = folder + '/depthMapBack.png'
    print("Saving Depth Map to:")
    print(depthMapPath)    

    cv2.imwrite(depthMapPath, rend_img_vp2)
    # cv2.imshow('Depth Map',rend_img_vp2)        
    # cv2.waitKey(0)

    # cv2.destroyAllWindows()


def preprocess_image(img_path, json_path=None):
    img = io.imread(img_path)
    if img.shape[2] == 4:
        img = img[:, :, :3]

    if json_path is None:
        if np.max(img.shape[:2]) != config.img_size:
            print('Resizing so the max image size is %d..' % config.img_size)
            scale = (float(config.img_size) / np.max(img.shape[:2]))
        else:
            scale = 1.
        center = np.round(np.array(img.shape[:2]) / 2).astype(int)
        # image center in (x,y)
        center = center[::-1]
    else:
        scale, center = op_util.get_bbox(json_path)

    crop, proc_param = img_util.scale_and_crop(img, scale, center,
                                               config.img_size)

    # Normalize image to [-1, 1]
    crop = 2 * ((crop / 255.) - 0.5)

    return crop, proc_param, img

In [4]:
config = flags.FLAGS

remaining_args = config([sys.argv[0]] + [flag for flag in sys.argv if flag.startswith("--")])
assert(remaining_args == [sys.argv[0]])

config.load_path = src.config.PRETRAINED_MODEL

config.batch_size = 1

In [5]:
sess = tf.Session()
model = RunModel(config, sess=sess)

W0319 23:53:10.383591  6564 deprecation_wrapper.py:119] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR_Modified\TensorFlow\src\RunModel.py:42: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0319 23:53:15.635912  6564 deprecation_wrapper.py:119] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR_Modified\TensorFlow\src\models.py:77: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.



Iteration 0


W0319 23:53:16.405761  6564 deprecation.py:323] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR_Modified\TensorFlow\src\tf_smpl\batch_lbs.py:55: div (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Iteration 1
Reuse is on!
Iteration 2
Reuse is on!


W0319 23:53:19.016145  6564 deprecation_wrapper.py:119] From C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR_Modified\TensorFlow\src\RunModel.py:70: The name tf.train.Saver is deprecated. Please use tf.compat.v1.train.Saver instead.

W0319 23:53:19.502724  6564 deprecation.py:323] From C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\training\saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Restoring checkpoint C:\_Files\MyProjects\ASDS_3\Photo_Wake-Up\src\HMR_Modified\TensorFlow\src\..\models\model.ckpt-667589..


In [6]:
img_path=config.img_path
json_path=config.json_path

input_img, proc_param, img = preprocess_image(img_path, json_path)
# Add batch dimension: 1 x D x D x 3
input_img = np.expand_dims(input_img, 0)


joints, verts, cams, joints3d, theta = model.predict(input_img, get_theta=True)

Resizing so the max image size is 224..


In [8]:
# visualize(img, proc_param, joints[0], verts[0], cams[0], img_path)

In [9]:
theta_out = theta.tolist()
with open('results/HMR_value_out.json', 'w') as outfile:
    json.dump([theta_out], outfile)